In [1]:
!pip install google-cloud-storage

/bin/sh: 1: pip: not found


In [2]:
import csv
import numpy as np
import pandas as pd
import subprocess
import requests
import json
import difflib
from google.cloud import storage

In [3]:
# Membaca file CSV dan menyimpannya di dalam variabel top_predictions
top_predictions = []
with open("Hasil_Rekomendasi.csv", mode='r') as file:
    reader = csv.reader(file)
    next(reader)  # Mengabaikan baris header
    for row in reader:
        predicted_label = row[0]
        probability = float(row[1])
        top_predictions.append((predicted_label, probability))

# Mendapatkan waktu dari system log Linux
# Pastikan Anda telah menginstal library subprocess sebelumnya
syslog_time = subprocess.check_output('date', shell=True).decode().strip()
time = syslog_time[11:16]
print(time)
#print(syslog_time)

location_url = 'https://ipgeolocation.abstractapi.com/v1/?api_key=c97e48fe3fae431f97b3167c7ef5df3b' 
response = requests.get(location_url)
location_data = response.json()

lon = str(location_data['longitude'])
lat = str(location_data['latitude'])
print("Latitude:", lat)
print("Longitude:", lon)
print ("-----------------------------------------------------------------------------------------------------")
#Weather
api_key = '83f803896e21d5c0975423db9e58d6f5'
weather_url = 'https://api.openweathermap.org/data/2.5/weather?lat='+lat+'&'+'lon='+lon+'&appid='+api_key+'&units=metric' 
response = requests.get(weather_url)
weather_data = response.json()
temp = weather_data['main']['temp']
print("Temp:", temp)

21:00
Latitude: -6.2114
Longitude: 106.8446
-----------------------------------------------------------------------------------------------------
Temp: 25.27


In [4]:
# Membaca data dari ListMakanan.csv
ListMakanan = pd.read_csv('FinalFood.csv', sep=';')

print(ListMakanan)

              A            B          C              D        E          F  \
0    Air dingin  Ayam geprek   Gorengan  Pisang goreng    Bakso       Buah   
1          Roti        Bakso   Gorengan    Sop kambing  Keripik  Sate ayam   
2        Es teh        Snack     Cireng        Brownis    Bakso     Pempek   
3       Indomie     Mie ayam  Nasi ayam     Cream soup      Sup       Buah   
4         Bubur        Bakso    Brownis          Steak      Sup      Cakwe   
..          ...          ...        ...            ...      ...        ...   
188         NaN          NaN        NaN            NaN      NaN        NaN   
189         NaN          NaN        NaN            NaN      NaN        NaN   
190         NaN          NaN        NaN            NaN      NaN        NaN   
191         NaN          NaN        NaN            NaN      NaN        NaN   
192         NaN          NaN        NaN            NaN      NaN        NaN   

            G              H             I             J       

In [5]:
# Mendapatkan kolom terpilih berdasarkan waktu dan variabel cuaca
selected_column = None

# Membentuk rentang waktu dan variabel cuaca yang sesuai dengan kolom terpilih
if '06:01' <= time <= '09:00' and temp >= 25:
    selected_column = 'A'
elif '06:01' <= time <= '09:00' and temp < 25:
    selected_column = 'B'
elif '09:01' <= time <= '12:00' and temp >= 25:
    selected_column = 'C'
elif '09:01' <= time <= '12:00' and temp >= 25:
    selected_column = 'D'
elif '12:01' <= time <= '15:00' and temp < 25:
    selected_column = 'E'
elif '12:01' <= time <= '15:00' and temp >= 25:
    selected_column = 'F'
elif '15:01' <= time <= '18:00' and temp < 25:
    selected_column = 'G'
elif '15:01' <= time <= '18:00' and temp >= 25:
    selected_column = 'H'
elif '18:01' <= time <= '21:00' and temp >= 25:
    selected_column = 'I'
elif '18:01' <= time <= '21:00' and temp < 25:
    selected_column = 'J'
elif '21:01' <= time <= '23:59' and temp >= 25:
    selected_column = 'K'
elif '21:01' <= time <= '23:59' and temp < 25:
    selected_column = 'L'
elif '00:00' <= time <= '03:00' and temp >= 25:
    selected_column = 'M'
elif '00:00' <= time <= '03:00' and temp < 25:
    selected_column = 'N'
elif '03:01' <= time <= '06:00' and temp >= 25:
    selected_column = 'O'
elif '03:01' <= time <= '06:00' and temp < 25:
    selected_column = 'P'

In [6]:
# Menginisialisasi daftar rekomendasi yang cocok dengan kolom terpilih
matched_recommendations = []
nama_resto = []

# Membandingkan rekomendasi dengan data pada kolom terpilih menggunakan cosine similarity
selected_data = [item for item in ListMakanan[selected_column].dropna().values]

# Jika tidak ada kesamaan penuh, tambahkan prediksi selanjutnya hingga mencapai 5 kesamaan
i = 5

while len(matched_recommendations) < 5:
    remaining_predictions = top_predictions[:i]
    i += 5
    remainings = [prediction[0] for prediction in remaining_predictions]
    for food_name in remainings:
        for data in selected_data:
            similarity_score = difflib.SequenceMatcher(None, food_name.lower(), data.lower()).ratio()
            if similarity_score >= 0.8:
                matched_recommendations.append(data)

    matched_recommendations = list(set(matched_recommendations))
    
    # Jika semua 5 rekomendasi cocok dengan data kolom terpilih, simpan nama makanan langsung
    if len(matched_recommendations) == 5:
        recommended_foods = matched_recommendations
        break  # Keluar dari loop while jika sudah ada 5 rekomendasi yang cocok

print(matched_recommendations)

['Latte ', 'Brownis ', 'Bubur ', 'Ayam Sambal Matah ', 'Croissant ']


In [7]:
# Membaca data dari DataResto.csv
RekomendasiAkhir = pd.read_csv('DataResto.csv', sep=',')
print(RekomendasiAkhir)

# Menginisialisasi daftar rekomendasi yang cocok dengan kolom terpilih
rekomendasi_akhir = []

# Membandingkan rekomendasi dengan data pada kolom terpilih menggunakan cosine similarity
selected_row = [item for item in RekomendasiAkhir['Nama_makanan'].dropna().values]

                                      Nama_resto  \
0      Warkop Berkah Big's Family Pasadena Depok   
1      Warkop Berkah Big's Family Pasadena Depok   
2      Warkop Berkah Big's Family Pasadena Depok   
3      Warkop Berkah Big's Family Pasadena Depok   
4      Warkop Berkah Big's Family Pasadena Depok   
...                                          ...   
31951        Nasi Goreng Sibungsu  - Kebon Sirih   
31952        Nasi Goreng Sibungsu  - Kebon Sirih   
31953        Nasi Goreng Sibungsu  - Kebon Sirih   
31954        Nasi Goreng Sibungsu  - Kebon Sirih   
31955        Nasi Goreng Sibungsu  - Kebon Sirih   

                                   Nama_makanan  
0                      Pisang Bakar Coklat Keju  
1                          Bubur Goreng Komplit  
2         Indomie Goreng Tek Tek (Tumis Kering)  
3      Indomie Nyemek Spesial (sedikit Berkuah)  
4                     Indomie Dog Dog (Berkuah)  
...                                         ...  
31951                    

In [8]:
i = 5

while len(rekomendasi_akhir) < 5:
    prediction_row = selected_row[:i]
    i += 5
    for name in matched_recommendations:
        for data in prediction_row:
            similarity_score = difflib.SequenceMatcher(None, food_name.lower(), data.lower()).ratio()
            if similarity_score >= 0.2:
                # Menyimpan Nama_resto dari baris yang sesuai dengan data
                row_index = RekomendasiAkhir[RekomendasiAkhir['Nama_makanan'] == data].index[0]
                nama_resto.append(RekomendasiAkhir.loc[row_index, 'Nama_resto'])
                rekomendasi_akhir.append(data)
    
    rekomendasi_akhir = list(set(rekomendasi_akhir))
    
    # Jika semua 5 rekomendasi cocok dengan data kolom terpilih, simpan nama makanan langsung
    if len(rekomendasi_akhir) == 5:
        final_recommendation = rekomendasi_akhir
        break  # Keluar dari loop while jika sudah ada 5 rekomendasi yang cocok
    elif i > len(RekomendasiAkhir):  # Memeriksa jika sudah mencapai row paling bawah
        break
# Menghapus elemen duplikat dari nama_resto
nama_resto = list(set(nama_resto))

# Menggabungkan semua baris dengan Nama_resto yang sama
matching_rows = RekomendasiAkhir[RekomendasiAkhir['Nama_resto'].isin(nama_resto)]

print(nama_resto)
print(matching_rows)

matching_rows.to_json('RekomendasiAkhir.json', orient='records')

["Warkop Berkah Big's Family Pasadena Depok"]
                                    Nama_resto  \
0    Warkop Berkah Big's Family Pasadena Depok   
1    Warkop Berkah Big's Family Pasadena Depok   
2    Warkop Berkah Big's Family Pasadena Depok   
3    Warkop Berkah Big's Family Pasadena Depok   
4    Warkop Berkah Big's Family Pasadena Depok   
..                                         ...   
278  Warkop Berkah Big's Family Pasadena Depok   
279  Warkop Berkah Big's Family Pasadena Depok   
280  Warkop Berkah Big's Family Pasadena Depok   
281  Warkop Berkah Big's Family Pasadena Depok   
282  Warkop Berkah Big's Family Pasadena Depok   

                                 Nama_makanan  
0                    Pisang Bakar Coklat Keju  
1                        Bubur Goreng Komplit  
2       Indomie Goreng Tek Tek (Tumis Kering)  
3    Indomie Nyemek Spesial (sedikit Berkuah)  
4                   Indomie Dog Dog (Berkuah)  
..                                        ...  
278              

In [9]:
# Mendefinisikan path ke file key service account
key_path = 'capstone-food-recommendation-d639e106b8c9.json'

# Membuat objek client storage
client = storage.Client.from_service_account_json(key_path)

# Mendefinisikan bucket GCP yang akan digunakan
bucket_name = 'findfud_bucket'

# Mengupload file ke bucket
blob = client.bucket(bucket_name).blob('RekomendasiAkhir.json')
blob.upload_from_filename('RekomendasiAkhir.json')